<a href="https://colab.research.google.com/github/JeffersonEspinalA/Trabajo-Final---IA/blob/main/C%C3%B3digo_fuente_Hito_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importar librerias

In [ ]:
import tensorflow
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras import optimizers
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras_preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD

Cargar datos

In [ ]:
dataset = pd.read_csv('drive/MyDrive/HAM10000/HAM10000_metadata.csv')
dataset['image_full_name'] = dataset['image_id']+'.jpg'

Creación del modelo

In [ ]:
modelo = Sequential()
modelo.add(Conv2D(64, (3, 3), activation='relu'))
modelo.add(MaxPooling2D(pool_size=(2, 2)))
modelo.add(Dropout(0.5))
modelo.add(Flatten())
modelo.add(Dense(128, activation='relu'))
modelo.add(Dropout(0.5))
modelo.add(Dense(7, activation='sigmoid'))

Dividir datos: entrenamiento, prueba y validación

In [ ]:
X = dataset[['image_full_name','dx','lesion_id']]
Y = X.pop('dx').to_frame()
X_entre, X_prueba, Y_entre, Y_prueba = train_test_split(X, Y, test_size=0.15, random_state=42)
X_entre, X_val, Y_entre, Y_val = train_test_split(X_entre, Y_entre, test_size=0.15, random_state=42)

In [ ]:
entre = pd.concat([X_entre, Y_entre],axis=1)
val = pd.concat([X_val, Y_val],axis=1)
prueba = pd.concat([X_prueba, Y_prueba],axis=1)

In [ ]:
encoder = LabelEncoder()
encoder.fit(val['dx'])
tipo_indice_entre = encoder.transform(val['dx']) 
val['label'] = tipo_indice_entre

In [ ]:
encoder = LabelEncoder()
encoder.fit(prueba['dx'])
tipo_indice_prueba = encoder.transform(prueba['dx']) 
prueba['label'] = tipo_indice_prueba

In [ ]:
entre_generator = ImageDataGenerator(rescale=1./255, rotation_range=10, zoom_range = 0.1, 
                                     width_shift_range=0.0, height_shift_range=0.00)

In [ ]:
datos_entre = entre_generator.flow_from_dataframe(dataframe=entre, x_col="image_full_name", y_col="dx",
                                                  batch_size=32, directory="drive/MyDrive/HAM10000/HAM10000_images",
                                                  shuffle=True, class_mode="categorical", target_size=(224,224))

Found 7235 validated image filenames belonging to 7 classes.


In [ ]:
prueba_generator = ImageDataGenerator(rescale=1./255,rotation_range=10, zoom_range=0.1, 
                                      width_shift_range=0.0, height_shift_range=0.02)

In [ ]:
datos_prueba = prueba_generator.flow_from_dataframe(dataframe=prueba, x_col="image_full_name", y_col="dx",
                                                    directory="drive/MyDrive/HAM10000/HAM10000_images",
                                                    shuffle=False, batch_size=1, class_mode=None, target_size=(224,224))

Found 1503 validated image filenames.


In [ ]:
datos_val = prueba_generator.flow_from_dataframe(dataframe=val, x_col="image_full_name", y_col="dx",
                                                 directory="drive/MyDrive/HAM10000/HAM10000_images",
                                                 batch_size=64, shuffle=False, class_mode="categorical", target_size=(224,224))

Found 1277 validated image filenames belonging to 7 classes.


Compilar modelo

In [ ]:
sgd = optimizers.SGD(learning_rate=0.01, clipvalue=0.5)
modelo.compile(optimizer=sgd,loss="categorical_crossentropy",metrics=["accuracy"])

Entrenar modelo

In [ ]:
history = modelo.fit_generator(generator=datos_entre, steps_per_epoch=datos_entre.samples//datos_entre.batch_size,
                               validation_data=datos_val, verbose=1, validation_steps=datos_val.samples//datos_val.batch_size,
                               epochs=3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  This is separate from the ipykernel package so we can avoid doing imports until


Epoch 1/3
226/226 [==============================] - 3937s 17s/step - loss: 1.2071 - accuracy: 0.6640 - val_loss: 1.0440 - val_accuracy: 0.6752
Epoch 2/3
226/226 [==============================] - 557s 2s/step - loss: 1.0299 - accuracy: 0.6706 - val_loss: 0.9667 - val_accuracy: 0.6752
Epoch 3/3
226/226 [==============================] - 551s 2s/step - loss: 0.9918 - accuracy: 0.6697 - val_loss: 0.9322 - val_accuracy: 0.6752


Evaluación porcentaje de precisión

In [ ]:
datos_prueba.reset()
predicciones = modelo.predict_generator(datos_prueba, steps=datos_prueba.samples/datos_prueba.batch_size, verbose=1)
x_pred = np.argmax(predicciones, axis=1)
cant = np.where(x_pred == tipo_indice_prueba)
print(f"Tasa de precision del modelo: {(np.count_nonzero(cant) / len(tipo_indice_prueba)) * 100}%")

   1/1503 [..............................] - ETA: 3:12

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  


1503/1503 [==============================] - 85s 57ms/step
Tasa de precision del modelo: 66.40053226879574%


Probar con una imagen en especifico

In [ ]:
def predecir(direccion_img):
  lesion_clases_dicc = {
    0 : 'Melanocytic nevi',
    1 : 'Melanoma',
    2 : 'Benign keratosis-like lesions ',
    3 : 'Basal cell carcinoma',
    4 : 'Actinic keratoses',
    5 : 'Vascular lesions',
    6 : 'Dermatofibroma'
  }

  imagen = tensorflow.keras.preprocessing.image.load_img(direccion_img, target_size=(224,224,3))

  x = tensorflow.keras.preprocessing.image.img_to_array(imagen)
  x = np.expand_dims(x, axis=0)

  prediccion = modelo.predict(x)
  pred_clase = prediccion.argmax(axis=-1)      

  pr = lesion_clases_dicc[pred_clase[0]]
  resultado =str(pr)  
  return resultado

In [ ]:
print(predecir("drive/MyDrive/HAM10000/HAM10000_images/ISIC_0029562.jpg"))

1/1 [==============================] - 0s 65ms/step
Melanoma


In [ ]:
print(predecir("drive/MyDrive/HAM10000/HAM10000_images/ISIC_0025775.jpg"))

1/1 [==============================] - 0s 150ms/step
Benign keratosis-like lesions 


In [ ]:
print(predecir("drive/MyDrive/HAM10000/HAM10000_images/ISIC_0032115.jpg"))

1/1 [==============================] - 0s 104ms/step
Melanocytic nevi


In [ ]:
print(predecir("drive/MyDrive/HAM10000/HAM10000_images/ISIC_0033883.jpg"))

1/1 [==============================] - 0s 67ms/step
Melanoma


In [ ]:
print(predecir("drive/MyDrive/HAM10000/HAM10000_images/ISIC_0031919.jpg"))

1/1 [==============================] - 0s 82ms/step
Melanocytic nevi


Guardar modelo

In [ ]:
modelo.save('Trabajo_Final.h5')